In [304]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
conn = sqlite3.connect('example.db')

In [31]:
conn.execute('''DROP table accounts''')


OperationalError: database table is locked

In [277]:
conn.execute('''CREATE TABLE accounts_ONE (account_id INT, gender TEXT, birthday DATE)''')


In [278]:
conn.execute('''CREATE TABLE weigh_ins_ONE
             (account_id INT, weighed_at TIMESTAMP, weight FLOAT)''')

In [279]:
conn.execute('''INSERT INTO accounts_ONE VALUES (1111,'M','1992-01-01')''')
conn.commit()

In [280]:
conn.execute('''INSERT INTO accounts_ONE VALUES (2222,'F','1882-01-01')''')
conn.commit()

In [281]:
conn.execute('''INSERT INTO accounts_ONE VALUES (3333,'M','1999-01-01')''')
conn.commit()

In [282]:
conn.execute('''INSERT INTO accounts_ONE VALUES (4444,'M','1960-01-01')''')
conn.commit()

In [283]:
conn.execute('''INSERT INTO weigh_ins_ONE VALUES (1111,'2017-02-01 10:15:12','90')''')
conn.commit()

In [284]:
conn.execute('''INSERT INTO weigh_ins_ONE VALUES (2222,'2017-02-10 1:15:12','100')''')
conn.commit()

In [285]:
conn.execute('''INSERT INTO weigh_ins_ONE VALUES (3333,'2016-01-10 6:15:12','180')''')
conn.commit()

In [286]:
conn.execute('''INSERT INTO weigh_ins_ONE VALUES (4444,'2016-09-10 3:15:12','390')''')
conn.commit()

In [275]:
conn.execute('''DROP table weigh_ins_ONE''')
conn.commit()

In [276]:
conn.execute('''DROP table accounts_ONE''')
conn.commit()

In [287]:
for row in conn.execute('SELECT * FROM accounts_ONE'):
        print (row)

(1111, 'M', '1992-01-01')
(2222, 'F', '1882-01-01')
(3333, 'M', '1999-01-01')
(4444, 'M', '1960-01-01')


In [288]:
for row in conn.execute('SELECT * FROM weigh_ins_ONE'):
        print (row)

(1111, '2017-02-01 10:15:12', 90.0)
(2222, '2017-02-10 1:15:12', 100.0)
(3333, '2016-01-10 6:15:12', 180.0)
(4444, '2016-09-10 3:15:12', 390.0)


In [289]:
t = conn.execute('''SELECT gender, COUNT(*) as num_records FROM accounts_ONE
JOIN weigh_ins_ONE ON accounts_ONE.account_id = weigh_ins_ONE.account_id GROUP BY gender''')



In [290]:
list(t)

[('F', 1), ('M', 3)]

In [291]:
df = pd.read_sql_query('''SELECT gender, COUNT(*) as num_accounts FROM accounts_ONE
JOIN weigh_ins_ONE ON accounts_ONE.account_id = weigh_ins_ONE.account_id GROUP BY gender''', conn)

In [292]:
df

,gender,num_accounts
0,F,1
1,M,3


In [293]:
df_one = pd.read_sql_query("""SELECT 
sum(CASE 
WHEN strftime('%Y','now') - birthday >=18 AND strftime('%Y','now') - birthday <=29 then 1 else 0 end) as '18-29',
sum(CASE
WHEN strftime('%Y','now') - birthday >=30 AND strftime('%Y','now') - birthday <=39 then 1 else 0 end) as '30-39',
sum(CASE
WHEN strftime('%Y','now') - birthday >=40 AND strftime('%Y','now') - birthday <=49 then 1 else 0 end) as '40-49',
sum(CASE
WHEN strftime('%Y','now') - birthday >=50 then 1 else 0 end) as '50+'
FROM accounts_ONE
 """,conn)

In [294]:
df_one

,18-29,30-39,40-49,50+
0,2,0,0,2


In [295]:
#SELECT strftime('%s','now') - strftime('%s','2004-01-01 02:34:56');

In [296]:
df_two = pd.read_sql_query("""SELECT 
sum(CASE 
WHEN strftime('%Y','now') - birthday >=18 AND strftime('%Y','now') - birthday <=29 then 1 else 0 end) as '18-29',
sum(CASE
WHEN strftime('%Y','now') - birthday >=30 AND strftime('%Y','now') - birthday <=39 then 1 else 0 end) as '30-39',
sum(CASE
WHEN strftime('%Y','now') - birthday >=40 AND strftime('%Y','now') - birthday <=49 then 1 else 0 end) as '40-49',
sum(CASE
WHEN strftime('%Y','now') - birthday >=50 then 1 else 0 end) as '50+'
FROM accounts_ONE
JOIN 
weigh_ins_ONE 
ON accounts_ONE.account_id = weigh_ins_ONE.account_id 
WHERE
weighed_at >= datetime('now','-14 days')
 """,conn)

In [297]:
df_two

,18-29,30-39,40-49,50+
0,1,0,0,1


In [408]:

account_id=[1,1,1,2,2,3,4,5]
item_id = [1,2,3,2,3,1,4,5]
rating = [3,3,1,1,1,3,1,2]

In [327]:
names_df = ['account_id','item_id','rating']

In [409]:
df_transform = pd.DataFrame(data=[account_id,item_id,rating]).T
df_transform.columns=names_df

In [410]:
df_transform

,account_id,item_id,rating
0,1,1,3
1,1,2,3
2,1,3,1
3,2,2,1
4,2,3,1
5,3,1,3
6,4,4,1
7,5,5,2


In [555]:
df_grouped_index = df_transform.set_index(keys=['account_id','item_id','rating']).T

df_grouped_index

Empty DataFrame
Columns: [(1, 1, 3), (1, 2, 3), (1, 3, 1), (2, 2, 1), (2, 3, 1), (3, 1, 3), (4, 4, 1), (5, 5, 2)]
Index: []

In [586]:
munged_df = pd.DataFrame(columns=df_transform.account_id.unique(),
                         index=[i+1 for i in range(max(df_transform.item_id))]) ## create a new DataFrame with null vals
for row, col in munged_df.iteritems():
    for col_num,item in enumerate(col):
        col_num = col_num+1 # Since the accounts start at 1 and not 0
        try: ## see if we have a rating for this item
            munged_df.loc[row, col_num] = list(df_grouped_index[col_num,row])[0]
        except KeyError: ## We don't have a rating
            continue
munged_df = munged_df.T ## Transpose to have accounts for the index and items for the columns
        
        

In [587]:
munged_df

,1,2,3,4,5
1,3,3,1,NaN,NaN
2,NaN,1,1,NaN,NaN
3,3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1,NaN
5,NaN,NaN,NaN,NaN,2


In [418]:
pd.DataFrame(df_grouped)

KeyError: 0

In [360]:
print([i for i in df_grouped.groups])

[1, 2, 3, 4, 5]


In [368]:
df_grouped.apply(lambda x: x if  df_transform.item_id.any() != np.nan else 0)

,account_id,item_id,rating
0,1,1,3
1,1,2,3
2,1,3,1
3,2,2,1
4,2,3,1
5,3,1,3
6,4,4,1
7,5,5,2
